In [30]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
import h2o
from h2o.automl import H2OAutoML

X=pd.read_csv('../data/processed/X.csv')
Y=pd.read_csv('../data/processed/Y.csv')
X_predict=pd.read_csv('../data/processed/X_predict.csv')
df_predict=pd.read_csv('../data/processed/df_predict.csv')

In [31]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.1-internal" 2018-10-16; OpenJDK Runtime Environment (build 11.0.1-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 11.0.1-internal+0-adhoc..src, mixed mode)
  Starting server from /home/carpiero/miniconda3/envs/ml_env/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp38ufmk3m
  JVM stdout: /tmp/tmp38ufmk3m/h2o_carpiero_started_from_python.out
  JVM stderr: /tmp/tmp38ufmk3m/h2o_carpiero_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,30 days
H2O_cluster_name:,H2O_from_python_carpiero_ldi8hv
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.686 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [32]:
X_Y=pd.concat([X,Y], axis=1)


In [33]:
x= X.columns.to_list()
y = "price"
x

['depth',
 'table',
 'x',
 'y',
 'z',
 'cut_int',
 'color_int',
 'clarity_int',
 'raw_density',
 'carat_sqr']

In [34]:
# H2OAutoML(max_models=50,sort_metric='RMSE') 511.999	
# H2OAutoML(max_models=100,sort_metric='RMSE')    
# H2OAutoML(max_models=200,sort_metric='RMSE')
# H2OAutoML(max_models=400,max_runtime_secs=1000,sort_metric='RMSE')  516.046
# H2OAutoML(max_models=50,max_runtime_secs=1000,sort_metric='RMSE') 512.375
# H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE') 513,651

train_X_Y=h2o.H2OFrame(X_Y)
aml = H2OAutoML(max_models=20,max_runtime_secs=1000,sort_metric='RMSE')
aml.train(x=x, y=y, training_frame=train_X_Y)


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [36]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200910_142928,513.651,263837,263837,262.097,0.0914283
StackedEnsemble_BestOfFamily_AutoML_20200910_142928,518.074,268401,268401,267.929,0.0966308
GBM_2_AutoML_20200910_142928,522.062,272549,272549,271.672,0.0981279
GBM_grid__1_AutoML_20200910_142928_model_1,522.96,273487,273487,264.349,0.0887019
GBM_1_AutoML_20200910_142928,525.877,276546,276546,276.711,0.102055
GBM_4_AutoML_20200910_142928,525.924,276596,276596,266.836,0.0890773
GBM_3_AutoML_20200910_142928,533.603,284732,284732,275.268,0.0962902
XGBoost_grid__1_AutoML_20200910_142928_model_1,543.82,295741,295741,288.861,0.106659
GBM_5_AutoML_20200910_142928,543.934,295864,295864,274.98,0.0954681
XGBoost_3_AutoML_20200910_142928,547.254,299487,299487,292.555,0.109636


In [26]:
predict_X=h2o.H2OFrame(X_predict)
predictions_competition= aml.leader.predict(predict_X)
predictions_competition=predictions_competition.as_data_frame()


Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


In [27]:

submission = pd.DataFrame({'id': df_predict['id'], 'price': predictions_competition['predict']})
submission['id'] = submission['id'].astype('int64')

submission.to_csv('/home/carpiero/IRONHACK/kaggle/submission.csv', index=False)



In [28]:
h2o.cluster().shutdown()

H2O session _sid_8364 closed.


In [29]:
submission.describe()

,id,price
count,13485.000000,13485.000000
mean,6742.000000,4350.812358
std,3892.928525,3422.831532
min,0.000000,878.985531
25%,3371.000000,1865.688443
50%,6742.000000,3088.094167
75%,10113.000000,5431.912224
max,13484.000000,18118.032304
